In [1]:
%matplotlib inline

In [2]:
#from pycocotools.coco import COCO
import glob
import _init_paths
from fast_rcnn.config import cfg
from fast_rcnn.test import im_detect
from fast_rcnn.nms_wrapper import nms
from utils.timer import Timer
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import caffe, os, sys, cv2
import argparse
from fast_rcnn.config import cfg, cfg_from_file, cfg_from_list


def _get_ann_file():
        prefix = 'image_info' 
        return os.path.join("/root/data/coco/", 'annotations',
                        prefix + '_' + "test2014" + '.json')

   
'''
COCO = COCO(_get_ann_file())
cats = COCO.loadCats(COCO.getCatIds())
CLASSES  = tuple(['__background__'] + [c['name'] for c in cats])


print(CLASSES)
'''

"\nCOCO = COCO(_get_ann_file())\ncats = COCO.loadCats(COCO.getCatIds())\nCLASSES  = tuple(['__background__'] + [c['name'] for c in cats])\n\n\nprint(CLASSES)\n"

In [ ]:
def vis_detections(im, class_names, dets_list, thresh=0.5):
    """Draw detected bounding boxes."""
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')
    
    for class_name, dets in zip(class_names, dets_list):

        inds = np.where(dets[:, -1] >= thresh)[0]
        if len(inds) == 0:
            continue


        for i in inds:
            bbox = dets[i, :4]
            score = dets[i, -1]

            ax.add_patch(
                plt.Rectangle((bbox[0], bbox[1]),
                              bbox[2] - bbox[0],
                              bbox[3] - bbox[1], fill=False,
                              edgecolor='red', linewidth=3.5)
                )
            ax.text(bbox[0], bbox[1] - 2,
                    '{:s} {:.3f}'.format(class_name, score),
                    bbox=dict(facecolor='blue', alpha=0.5),
                    fontsize=14, color='white')

        ax.set_title(('{} detections with '
                      'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                      thresh),
                      fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

def demo(net, im_file):
    """Detect object classes in an image using pre-computed object proposals."""

    # Load the demo image
   
    im = cv2.imread(im_file)
    print(im.shape)
    #int("Hallo")
    

    # Detect all object classes and regress object bounds
    timer = Timer()
    timer.tic()
    _t = {'im_preproc': Timer(), 'im_net' : Timer(), 'im_postproc': Timer(), 'misc' : Timer()}
    scores, boxes = im_detect(net, im, _t)
    timer.toc()
    #print ('Detection took {:.3f}s for '
           #'{:d} object proposals').format(timer.total_time, boxes.shape[0])

    # Visualize detections for each class
    CONF_THRESH = 0.8
    NMS_THRESH = 0.3
    
    cls_list = []
    dets_list = []
    
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes,
                          cls_scores[:, np.newaxis])).astype(np.float32)
        keep = nms(dets, NMS_THRESH)
        dets = dets[keep, :]
        
        cls_list.append(cls)
        dets_list.append(dets)
        
        
    vis_detections(im, cls_list, dets_list, thresh=CONF_THRESH)



if __name__ == '__main__':
    cfg.TEST.HAS_RPN = True
   

    prototxt = "../models/pvanet/lite/coco_test.prototxt"
    caffemodel = "../models/coco/pvanet/all-coco80_10K.caffemodel"

    caffe.set_mode_gpu()
    caffe.set_device(3)
    cfg.GPU_ID = 3
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)

    print '\n\nLoaded network {:s}'.format(caffemodel)

    # Warmup on a dummy image
    #m = 128 * np.ones((300, 500, 3), dtype=np.uint8)
    #for i in xrange(2):
        #_t = {'im_preproc': Timer(), 'im_net' : Timer(), 'im_postproc': Timer(), 'misc' : Timer()}
        #_, _= im_detect(net, im, _t)
    
    im_names = ['/root/data/data-HighwayDay/images/set00/V000/set00_V000_689.jpg',\
                '/root/data/data-HighwayDay/images/set00/V000/set00_V000_789.jpg',\
                '/root/data/data-HighwayDay/images/set00/V000/set00_V000_889.jpg',\
                '/root/data/data-HighwayDay/images/set00/V000/set00_V000_989.jpg',\
                '/root/data/data-HighwayDay/images/set00/V000/set00_V000_1089.jpg']
    
    
    im_names += glob.glob("/root/data/demo/suitcase/*.jpg")
    
    
    for im_name in im_names:
        print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
        print 'Demo for data/demo/{}'.format(im_name)
        demo(net, im_name)

    plt.show()

In [3]:
%load_ext autoreload
%autoreload 2

from datasets.pascal_voc_new import pascal_voc
from datasets.coco import coco
from datasets.vatic import VaticData, IMDBGroup
import numpy as np





In [15]:

coco_train = coco("train", "2014")
coco_val = coco("val", "2014")
#global classes
classes = coco_val._classes
print(classes)
devkit_path = "/root/data/VOCdevkit"
mapper = {"tvmonitor":"tv", "sofa":"couch", "aeroplane":"airplane",\
          "motorbike":"motorcycle", "diningtable":"dining table", "pottedplant":"potted plant"}
voc2007 = pascal_voc(classes, "trainval", "2007", cls_mapper=mapper, devkit_path=devkit_path)
voc2012 = pascal_voc(classes, "trainval", "2012", cls_mapper=mapper, devkit_path=devkit_path)
vatic_names = ["YuDa","A1HighwayDay", "B2HighwayNight"]
mapper = {"van":"car", "trailer-head":"truck",\
          "sedan/suv":"car", "scooter":"motorcycle", "bike":"bicycle"}

vatics = [VaticData(vatic_name, "coco", CLS_mapper=mapper,  train_split="all") for vatic_name in vatic_names]
datasets = vatics + [coco_train, coco_val]  + [voc2007, voc2012] + vatics


imdb_group = IMDBGroup(datasets)

loading annotations into memory...
Done (t=18.12s)
creating index...
index created!
loading annotations into memory...
Done (t=4.25s)
creating index...
index created!
('__background__', u'person', u'bicycle', u'car', u'motorcycle', u'airplane', u'bus', u'train', u'truck', u'boat', u'traffic light', u'fire hydrant', u'stop sign', u'parking meter', u'bench', u'bird', u'cat', u'dog', u'horse', u'sheep', u'cow', u'elephant', u'bear', u'zebra', u'giraffe', u'backpack', u'umbrella', u'handbag', u'tie', u'suitcase', u'frisbee', u'skis', u'snowboard', u'sports ball', u'kite', u'baseball bat', u'baseball glove', u'skateboard', u'surfboard', u'tennis racket', u'bottle', u'wine glass', u'cup', u'fork', u'knife', u'spoon', u'bowl', u'banana', u'apple', u'sandwich', u'orange', u'broccoli', u'carrot', u'hot dog', u'pizza', u'donut', u'cake', u'chair', u'couch', u'potted plant', u'bed', u'dining table', u'toilet', u'tv', u'laptop', u'mouse', u'remote', u'keyboard', u'cell phone', u'microwave', u'oven

In [7]:
vatic_names = ["YuDa","A1HighwayDay", "B2HighwayNight"]
mapper = {"van":"car", "trailer-head":"truck",\
          "sedan/suv":"car", "scooter":"motorcycle", "bike":"bicycle"}
vatics = [VaticData(vatic_name, "coco", CLS_mapper=mapper,  train_split="all") for vatic_name in vatic_names]

Meta data path: /root/data/data-YuDa/meta.json does not exist. Use Default meta data
Use both split for training
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0, 1], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 28220 images
Meta data path: /root/data/data-A1HighwayDay/meta.json does not exist. Use Default meta data
Use both split for training
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0, 1], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 6320 images
Meta data path: /root/data/data-B2HighwayNight/meta.json does not exist. Use Default meta data
Use both split for training
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0, 1], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 6320 images


In [3]:
def viz_imdb(imdb, k=5): #visualize the image randomly of an imdb
    gt_roidb = imdb.gt_roidb()
    
    
    
    
    for i in np.random.choice(len(gt_roidb), k):
        im_path = imdb.image_path_at(i)
        im = cv2.imread(im_path)
        im = im[:, :, (2, 1, 0)]
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(im, aspect='equal')
        
        gt = gt_roidb[i]
        
        for n in range(len(gt["gt_classes"])):
        
            cls_ind = gt["gt_classes"][n]
            #print(cls_ind)
            label = imdb._datasets[0]._classes[cls_ind]
            bbox = gt["boxes"][n, :]

            if gt["gt_overlaps"][n, cls_ind] == 1:

                ax.add_patch(
                    plt.Rectangle((bbox[0], bbox[1]),
                                  bbox[2] - bbox[0],
                                  bbox[3] - bbox[1], fill=False,
                                  edgecolor='green', linewidth=3.5)
                    )

                ax.text(bbox[0], bbox[1] - 2,
                        '{:s}'.format(label),
                        bbox=dict(facecolor='blue', alpha=0.5),
                        fontsize=14, color='white')

        plt.axis('off')
        plt.tight_layout()
        
        
        
    plt.draw()



viz_imdb(imdb_group)

coco_2014_train gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_train_gt_roidb.pkl
coco_2014_val gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_val_gt_roidb.pkl
voc_2007_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl
voc_2012_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2012_trainval_gt_roidb.pkl


In [4]:
viz_imdb(imdb_group, k =20)

coco_2014_train gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_train_gt_roidb.pkl
coco_2014_val gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_val_gt_roidb.pkl
voc_2007_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl
voc_2012_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2012_trainval_gt_roidb.pkl


/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [28]:
len(imdb_group.gt_roidb())

coco_2014_train gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_train_gt_roidb.pkl
voc_2007_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl


108224

In [16]:
total_len = float(len(imdb_group.gt_roidb()))



for dataset in imdb_group._datasets:
    img_nums = len(dataset.gt_roidb())
   
    print(dataset.name, img_nums,  "{0:.2f}%".format(img_nums/total_len * 100))
#imdb_group._datasets[0].name

coco_2014_train gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_train_gt_roidb.pkl
coco_2014_val gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_val_gt_roidb.pkl
voc_2007_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl
voc_2012_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2012_trainval_gt_roidb.pkl
('vatic_YuDa', 28220, '12.74%')
('vatic_A1HighwayDay', 6320, '2.85%')
('vatic_B2HighwayNight', 6320, '2.85%')
coco_2014_train gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_train_gt_roidb.pkl
('coco_2014_train', 82783, '37.36%')
coco_2014_val gt roidb loaded from /root/pva-faster-rcnn/data/cache/coco_2014_val_gt_roidb.pkl
('coco_2014_val', 40504, '18.28%')
voc_2007_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2007_trainval_gt_roidb.pkl
('voc_2007_trainval', 5011, '2.26%')
voc_2012_trainval gt roidb loaded from /root/pva-faster-rcnn/data/cache/voc_2012_tra

In [17]:
total_len

221558.0

In [ ]:
from